In [1]:
import tensorflow as tf 
from tensorflow import keras
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#Load Datset
mnist = keras.datasets.mnist.load_data()

In [3]:
#Split dataset
from sklearn.model_selection import train_test_split
(X_train_full, y_train_full), (X_test, y_test) = mnist

In [4]:
#X_train_full = X_train_full.reshape(X_train_full.shape[0], 784)
#X_test = X_test.reshape(X_test.shape[0], 784)

In [5]:
X_train_full.shape

(60000, 28, 28)

In [6]:
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0

In [7]:
np.random.seed(42)
tf.random.set_seed(42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

In [8]:
def create_sequential(n_neurons=100, n_hidden=2, activation="relu", learning_rate=3e-1):
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=[28, 28]))
    for i in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation=activation))
    model.add(keras.layers.Dense(10, activation="softmax"))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

In [9]:
root_logdir = os.path.join(os.curdir, "my_logs")

def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)
run_logdir = get_run_logdir()

In [10]:
#Define all callbacks
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
checkpoint_cb = keras.callbacks.ModelCheckpoint("mnist_model.h5", save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

In [11]:
mnist_clf = keras.wrappers.scikit_learn.KerasClassifier(create_sequential)

In [17]:
#Setup RandomSearch
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "n_hidden": [1, 2],
}

rnd_search = RandomizedSearchCV(mnist_clf, param_distribs, n_iter=10, cv=3)
rnd_search.fit(X_train, y_train, epochs=100,
              validation_data=(X_valid, y_valid),
             callbacks=[early_stopping_cb, checkpoint_cb, tensorboard_cb])

/opt/homebrew/anaconda3/envs/tensorflow_env/lib/python3.7/site-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 2 is smaller than n_iter=10. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
/opt/homebrew/anaconda3/envs/tensorflow_env/lib/python3.7/site-packages/sklearn/model_selection/_search.py:269: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  random_state=rnd):
/opt/homebrew/anaconda3/envs/tensorflow_env/lib/python3.7/site-packages/

Train on 30000 samples, validate on 15000 samples
Epoch 1/100
30000/30000 [==============================] - 4s 121us/sample - loss: 0.3028 - accuracy: 0.9080 - val_loss: 0.1828 - val_accuracy: 0.9455
Epoch 2/100
30000/30000 [==============================] - 3s 105us/sample - loss: 0.1338 - accuracy: 0.9598 - val_loss: 0.1389 - val_accuracy: 0.9581
Epoch 3/100
30000/30000 [==============================] - 3s 104us/sample - loss: 0.0968 - accuracy: 0.9703 - val_loss: 0.1267 - val_accuracy: 0.9605
Epoch 4/100
30000/30000 [==============================] - 3s 104us/sample - loss: 0.0716 - accuracy: 0.9770 - val_loss: 0.1060 - val_accuracy: 0.9694
Epoch 5/100
30000/30000 [==============================] - 3s 105us/sample - loss: 0.0536 - accuracy: 0.9834 - val_loss: 0.1190 - val_accuracy: 0.9654
Epoch 6/100
30000/30000 [==============================] - 3s 105us/sample - loss: 0.0434 - accuracy: 0.9863 - val_loss: 0.1050 - val_accuracy: 0.9702
Epoch 7/100
30000/30000 [===================

Train on 30000 samples, validate on 15000 samples
Epoch 1/100
30000/30000 [==============================] - 4s 126us/sample - loss: 0.3074 - accuracy: 0.9066 - val_loss: 0.1759 - val_accuracy: 0.9459
Epoch 2/100
30000/30000 [==============================] - 3s 110us/sample - loss: 0.1375 - accuracy: 0.9585 - val_loss: 0.1436 - val_accuracy: 0.9563
Epoch 3/100
30000/30000 [==============================] - 3s 105us/sample - loss: 0.0991 - accuracy: 0.9698 - val_loss: 0.1177 - val_accuracy: 0.9632
Epoch 4/100
30000/30000 [==============================] - 3s 106us/sample - loss: 0.0743 - accuracy: 0.9768 - val_loss: 0.1194 - val_accuracy: 0.9655
Epoch 5/100
30000/30000 [==============================] - 3s 105us/sample - loss: 0.0579 - accuracy: 0.9818 - val_loss: 0.1162 - val_accuracy: 0.9663
Epoch 6/100
30000/30000 [==============================] - 3s 105us/sample - loss: 0.0454 - accuracy: 0.9857 - val_loss: 0.1175 - val_accuracy: 0.9675
Epoch 7/100
30000/30000 [===================

Train on 30000 samples, validate on 15000 samples
Epoch 1/100
30000/30000 [==============================] - 4s 122us/sample - loss: 0.3080 - accuracy: 0.9066 - val_loss: 0.1826 - val_accuracy: 0.9460
Epoch 2/100
30000/30000 [==============================] - 3s 108us/sample - loss: 0.1385 - accuracy: 0.9583 - val_loss: 0.1386 - val_accuracy: 0.9569
Epoch 3/100
30000/30000 [==============================] - 3s 107us/sample - loss: 0.0979 - accuracy: 0.9710 - val_loss: 0.1455 - val_accuracy: 0.9527
Epoch 4/100
30000/30000 [==============================] - 3s 105us/sample - loss: 0.0745 - accuracy: 0.9780 - val_loss: 0.1081 - val_accuracy: 0.9676
Epoch 5/100
30000/30000 [==============================] - 3s 106us/sample - loss: 0.0589 - accuracy: 0.9825 - val_loss: 0.1079 - val_accuracy: 0.9679
Epoch 6/100
30000/30000 [==============================] - 3s 107us/sample - loss: 0.0459 - accuracy: 0.9859 - val_loss: 0.1119 - val_accuracy: 0.9692
Epoch 7/100
30000/30000 [===================

Train on 30000 samples, validate on 15000 samples
Epoch 1/100
30000/30000 [==============================] - 5s 166us/sample - loss: 0.3270 - accuracy: 0.8967 - val_loss: 0.1788 - val_accuracy: 0.9448
Epoch 2/100
30000/30000 [==============================] - 4s 139us/sample - loss: 0.1330 - accuracy: 0.9589 - val_loss: 0.1338 - val_accuracy: 0.9608
Epoch 3/100
30000/30000 [==============================] - 4s 149us/sample - loss: 0.0988 - accuracy: 0.9693 - val_loss: 0.1630 - val_accuracy: 0.9506
Epoch 4/100
30000/30000 [==============================] - 4s 143us/sample - loss: 0.0691 - accuracy: 0.9783 - val_loss: 0.1246 - val_accuracy: 0.9664
Epoch 5/100
30000/30000 [==============================] - 4s 142us/sample - loss: 0.0542 - accuracy: 0.9816 - val_loss: 0.1539 - val_accuracy: 0.9599
Epoch 6/100
30000/30000 [==============================] - 4s 141us/sample - loss: 0.0467 - accuracy: 0.9845 - val_loss: 0.1290 - val_accuracy: 0.9676
Epoch 7/100
30000/30000 [===================

...

Train on 30000 samples, validate on 15000 samples
Epoch 1/100
30000/30000 [==============================] - 5s 153us/sample - loss: 0.3374 - accuracy: 0.8945 - val_loss: 0.3746 - val_accuracy: 0.8805
Epoch 2/100
30000/30000 [==============================] - 4s 140us/sample - loss: 0.1428 - accuracy: 0.9553 - val_loss: 0.1365 - val_accuracy: 0.9597
Epoch 3/100
30000/30000 [==============================] - 4s 145us/sample - loss: 0.1018 - accuracy: 0.9675 - val_loss: 0.1699 - val_accuracy: 0.9458
Epoch 4/100
30000/30000 [==============================] - 4s 142us/sample - loss: 0.0791 - accuracy: 0.9751 - val_loss: 0.1177 - val_accuracy: 0.9683
Epoch 5/100
30000/30000 [==============================] - 4s 141us/sample - loss: 0.0614 - accuracy: 0.9799 - val_loss: 0.1345 - val_accuracy: 0.9618
Epoch 6/100
30000/30000 [==============================] - 4s 141us/sample - loss: 0.0485 - accuracy: 0.9842 - val_loss: 0.1551 - val_accuracy: 0.9596
Epoch 7/100
30000/30000 [===================

Train on 45000 samples, validate on 15000 samples
Epoch 1/100


/opt/homebrew/anaconda3/envs/tensorflow_env/lib/python3.7/site-packages/sklearn/model_selection/_search.py:793: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)


45000/45000 [==============================] - 5s 108us/sample - loss: 0.2629 - accuracy: 0.9194 - val_loss: 0.1635 - val_accuracy: 0.9538
Epoch 2/100
45000/45000 [==============================] - 4s 98us/sample - loss: 0.1162 - accuracy: 0.9651 - val_loss: 0.1124 - val_accuracy: 0.9655
Epoch 3/100
45000/45000 [==============================] - 5s 102us/sample - loss: 0.0829 - accuracy: 0.9750 - val_loss: 0.0942 - val_accuracy: 0.9711
Epoch 4/100
45000/45000 [==============================] - 4s 99us/sample - loss: 0.0636 - accuracy: 0.9807 - val_loss: 0.1017 - val_accuracy: 0.9709
Epoch 5/100
45000/45000 [==============================] - 4s 97us/sample - loss: 0.0510 - accuracy: 0.9841 - val_loss: 0.0981 - val_accuracy: 0.9706
Epoch 6/100
45000/45000 [==============================] - 4s 97us/sample - loss: 0.0403 - accuracy: 0.9871 - val_loss: 0.0848 - val_accuracy: 0.9746
Epoch 7/100
45000/45000 [==============================] - 4s 98us/sample - loss: 0.0317 - accuracy: 0.9905 - 

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fab86423450>,
                   iid='warn', n_iter=10, n_jobs=None,
                   param_distributions={'n_hidden': [1, 2]},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring=None, verbose=0)

In [18]:
rnd_search.best_params_

{'n_hidden': 1}

In [21]:
y_pred = rnd_search.best_estimator_.predict(X_test)

In [22]:
from sklearn.metrics import accuracy_score

In [23]:
accuracy_score(y_test, y_pred)

0.9775